# FPL Advanced Metrics & Analysis Demo

This notebook demonstrates **20+ advanced metrics and strategies** used by popular FPL content creators:
- FPL Focal
- Let's Talk FPL
- FPL Wire
- FPL General
- FPL BlackBox
- FPL Raptor

## Features:
1. Expected Goals (xG) and Expected Assists (xA) analysis
2. Position-specific metrics (GKP, DEF, MID, FWD)
3. Team building strategies
4. Transfer planning
5. Chip strategy (Wildcard, Bench Boost, Triple Captain, Free Hit)
6. Captain picks
7. Differential players
8. Fixture analysis
9. And much more!

## Setup & Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from Fpl_api import *
from advanced_metrics import AdvancedMetrics, FixtureAnalysis, load_fpl_data, generate_metric_summary
from fpl_strategy import FPLStrategy, GameweekPlanner, BonusPredictor

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)

# Plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

In [ ]:
# Load FPL data from API
print("Fetching latest FPL data...")
fpl_data = FPLapi_main_endpoint()

# Extract players and teams
players_df = players(fpl_data)
teams_df = teamdata(fpl_data)

print(f"\n✅ Loaded {len(players_df)} players and {len(teams_df)} teams")
print(f"\nData columns: {list(players_df.columns[:10])}...")

## 📊 Part 1: General Advanced Metrics

### Metrics used by all popular FPL creators

In [ ]:
# Initialize Advanced Metrics calculator
metrics = AdvancedMetrics(players_df)

# Display available metrics
print(generate_metric_summary())

In [ ]:
# Get comprehensive analysis
analysis_df = metrics.get_comprehensive_analysis()

# Top 20 players by value (Points per Million)
print("\n🏆 TOP 20 PLAYERS BY VALUE (Points per Million)")
print("=" * 80)
top_value = analysis_df.nlargest(20, 'points_per_million')[[
    'web_name', 'position', 'team', 'now_cost', 'total_points', 
    'points_per_million', 'form', 'xgi_per_90'
]]
print(top_value.to_string(index=False))

In [ ]:
# Expected Goal Involvements (xGI) Leaders
print("\n⚽ TOP 20 PLAYERS BY xGI PER 90")
print("Expected Goal Involvements - Key metric used by FPL Focal")
print("=" * 80)

top_xgi = analysis_df[analysis_df['minutes'] >= 500].nlargest(20, 'xgi_per_90')[[
    'web_name', 'position', 'team', 'now_cost', 
    'xgi_per_90', 'total_points', 'form'
]]
print(top_xgi.to_string(index=False))

In [ ]:
# Bonus Point Potential
print("\n🎁 TOP 20 BONUS POINT MAGNETS")
print("BPS per 90 - Players who consistently get bonus points")
print("=" * 80)

bonus_df = analysis_df[analysis_df['minutes'] >= 500].copy()
top_bonus = bonus_df.nlargest(20, 'bps_per_90')[[
    'web_name', 'position', 'team', 'now_cost',
    'bps_per_90', 'total_points', 'form'
]]
print(top_bonus.to_string(index=False))

## 🎯 Part 2: Position-Specific Analysis

### Forward Metrics

In [ ]:
# Forward-specific metrics
forwards = metrics.calculate_forward_metrics()

print("\n⚡ TOP FORWARDS ANALYSIS")
print("Shot Quality, Conversion Rate, Penalty Box Threat")
print("=" * 80)

if not forwards.empty:
    top_forwards = forwards.nlargest(15, 'forward_score')
    print(top_forwards.to_string(index=False))
else:
    print("No forward data available")

### Midfielder Metrics

In [ ]:
# Midfielder-specific metrics
midfielders = metrics.calculate_midfielder_metrics()

print("\n🎨 TOP MIDFIELDERS ANALYSIS")
print("Creativity Index, xA per 90, Goal Involvement")
print("=" * 80)

if not midfielders.empty:
    top_mids = midfielders.nlargest(15, 'midfielder_score')
    print(top_mids.to_string(index=False))
else:
    print("No midfielder data available")

### Defender Metrics

In [ ]:
# Defender-specific metrics
defenders = metrics.calculate_defender_metrics()

print("\n🛡️ TOP DEFENDERS ANALYSIS")
print("Clean Sheet Probability, Defensive Quality, Attacking Threat")
print("=" * 80)

if not defenders.empty:
    top_defs = defenders.nlargest(15, 'defender_score')
    print(top_defs.to_string(index=False))
else:
    print("No defender data available")

### Goalkeeper Metrics

In [ ]:
# Goalkeeper-specific metrics
goalkeepers = metrics.calculate_goalkeeper_metrics()

print("\n🧤 TOP GOALKEEPERS ANALYSIS")
print("Saves per 90, xG Prevented, Clean Sheet %")
print("=" * 80)

if not goalkeepers.empty:
    top_gks = goalkeepers.nlargest(10, 'goalkeeper_score')
    print(top_gks.to_string(index=False))
else:
    print("No goalkeeper data available")

## 👑 Part 3: Captain Analysis

### Best captaincy options based on form, xGI, and reliability

In [ ]:
# Captain picks
captain_picks = metrics.calculate_captain_score()

print("\n👑 TOP 15 CAPTAINCY OPTIONS")
print("Based on form, xGI, minutes reliability")
print("Technique used by Let's Talk FPL, FPL Wire")
print("=" * 80)

top_captains = captain_picks.head(15)
print(top_captains.to_string(index=False))

## 🔍 Part 4: Differential Players

### Low-owned gems with strong metrics

In [ ]:
# Find differentials
differentials = metrics.find_differentials(max_ownership=10.0, min_points=30)

print("\n💎 TOP DIFFERENTIAL PLAYERS")
print("Low ownership (<10%), strong underlying stats")
print("Strategy from FPL Wire, FPL BlackBox")
print("=" * 80)

if not differentials.empty:
    top_diffs = differentials.head(20)
    print(top_diffs.to_string(index=False))
else:
    print("No differentials found with current criteria")

## 📈 Part 5: Visualizations

### Visual analysis of key metrics

In [ ]:
# xGI vs Cost scatter plot
fig, ax = plt.subplots(figsize=(14, 8))

analysis_with_minutes = analysis_df[analysis_df['minutes'] >= 500].copy()

positions = analysis_with_minutes['position'].unique()
colors = {'GKP': 'yellow', 'DEF': 'blue', 'MID': 'green', 'FWD': 'red'}

for pos in positions:
    pos_data = analysis_with_minutes[analysis_with_minutes['position'] == pos]
    ax.scatter(
        pos_data['now_cost'] / 10,
        pos_data['xgi_per_90'],
        c=colors.get(pos, 'gray'),
        label=pos,
        alpha=0.6,
        s=100
    )

# Highlight top performers
top_performers = analysis_with_minutes.nlargest(10, 'xgi_per_90')
for _, player in top_performers.iterrows():
    ax.annotate(
        player['web_name'],
        (player['now_cost']/10, player['xgi_per_90']),
        fontsize=8,
        alpha=0.7
    )

ax.set_xlabel('Cost (£m)', fontsize=12)
ax.set_ylabel('xGI per 90', fontsize=12)
ax.set_title('Expected Goal Involvements vs Cost\n(Min 500 minutes)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Look for players above the trend line - good value for xGI!")

In [ ]:
# Points per Million by position
fig, ax = plt.subplots(figsize=(12, 6))

analysis_filtered = analysis_df[analysis_df['minutes'] >= 300].copy()

positions_order = ['FWD', 'MID', 'DEF', 'GKP']
data_by_position = [analysis_filtered[analysis_filtered['position'] == pos]['points_per_million'].dropna() 
                     for pos in positions_order]

bp = ax.boxplot(data_by_position, labels=positions_order, patch_artist=True)

# Color boxes
for patch, pos in zip(bp['boxes'], positions_order):
    patch.set_facecolor(colors.get(pos, 'gray'))
    patch.set_alpha(0.6)

ax.set_ylabel('Points per Million', fontsize=12)
ax.set_xlabel('Position', fontsize=12)
ax.set_title('Value Distribution by Position (Points per Million)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n💡 Midfielders and Forwards typically offer better PPM value!")

## 🎮 Part 6: Team Building Strategies

### Different team building approaches

In [ ]:
# Initialize strategy module
strategy = FPLStrategy(analysis_df)

print("\n🏗️ TEAM BUILDING STRATEGIES")
print("=" * 80)

# Build balanced team
print("\n1. BALANCED STRATEGY (Mix of price points)")
balanced_team = strategy.build_optimal_team(budget=100.0, formation="3-4-3", strategy="balanced")

print(f"\nStarters ({len(balanced_team['starters'])}):")
for player in balanced_team['starters']:
    print(f"  {player['name']:20} ({player['position']}) - £{player['cost']:.1f}m")

print(f"\nBench ({len(balanced_team['bench'])}):")
for player in balanced_team['bench']:
    print(f"  {player['name']:20} ({player['position']}) - £{player['cost']:.1f}m")

print(f"\nTotal Cost: £{balanced_team['total_cost']:.1f}m")

In [ ]:
# Differential team strategy
print("\n2. DIFFERENTIAL STRATEGY (Low ownership focus)")
diff_team = strategy.build_optimal_team(budget=100.0, formation="3-5-2", strategy="differential")

print(f"\nStarters ({len(diff_team['starters'])}):")
for player in diff_team['starters']:
    print(f"  {player['name']:20} ({player['position']}) - £{player['cost']:.1f}m")

print(f"\nTotal Cost: £{diff_team['total_cost']:.1f}m")
print("\n💡 Good for chasing in mini-leagues!")

## 🔄 Part 7: Transfer Planning

### Weekly transfer suggestions

In [ ]:
# Example: Analyze transfer targets
# Replace with your actual team
current_team = [
    'Saka', 'Salah', 'Palmer', 'Gordon', 'Bowen',  # Mids
    'Haaland', 'Watkins', 'Isak',  # Forwards
    'Gabriel', 'Pedro Porro', 'Saliba', 'Hall', 'Robinson',  # Defenders
    'Raya', 'Flekken'  # GKs
]

print("\n🔄 TRANSFER SUGGESTIONS")
print("Based on form, xGI, and fixtures")
print("=" * 80)

transfer_suggestions = strategy.analyze_transfer_targets(
    current_team=current_team,
    budget=2.0,  # £2.0m in the bank
    free_transfers=1,
    gameweeks_ahead=5
)

if transfer_suggestions:
    print(f"\nTop {len(transfer_suggestions[:5])} Transfer Recommendations:\n")
    for i, suggestion in enumerate(transfer_suggestions[:5], 1):
        print(f"{i}. {suggestion.player_out} ➡️  {suggestion.player_in} ({suggestion.position})")
        print(f"   Cost: {suggestion.cost_change:+.1f}m | Expected gain: {suggestion.expected_points_gain:.1f} pts")
        print(f"   Reason: {suggestion.reason}\n")
else:
    print("No transfer suggestions - team looks good!")

## 🎯 Part 8: Chip Strategy

### When to use your chips

In [ ]:
# Wildcard planning
current_gw = 15  # Update with current gameweek

print("\n🃏 CHIP STRATEGY GUIDE")
print("=" * 80)

# Wildcard
print("\n1️⃣ WILDCARD STRATEGY")
wc_plan = strategy.wildcard_planning(current_gw)
print(f"Current GW: {wc_plan['gameweek']}")
print(f"Window: {wc_plan['window']}")
print(f"Recommended: {'✅ Yes' if wc_plan['recommended'] else '❌ No'}")
print(f"Strategy: {wc_plan['strategy']}")
print(f"Reasoning: {wc_plan['reasoning']}")

In [ ]:
# Bench Boost
print("\n2️⃣ BENCH BOOST STRATEGY")
bb_plan = strategy.bench_boost_planner(upcoming_dgw=26)
print(f"Optimal timing: {bb_plan['optimal_timing']}")
print(f"Expected return: {bb_plan['expected_return']}")
print("\nPreparation steps:")
for step in bb_plan['preparation']:
    print(f"  {step}")

In [ ]:
# Triple Captain
print("\n3️⃣ TRIPLE CAPTAIN STRATEGY")
tc_plan = strategy.triple_captain_analysis(captain_picks)
print(f"Optimal timing: {tc_plan['optimal_timing']}")
print("\nTop targets:")
for target in tc_plan['top_targets'][:5]:
    print(f"  {target['name']} - Form: {target['form']:.1f}, xGI: {target['xgi_per_90']:.2f}")

In [ ]:
# Free Hit
print("\n4️⃣ FREE HIT STRATEGY")
fh_plan = strategy.free_hit_strategy(current_gw)
print(f"Optimal timing: {fh_plan['optimal_timing']}")
print(f"Typical GWs: {fh_plan['typical_gameweeks']}")
print("\nStrategy:")
for step in fh_plan['strategy']:
    print(f"  {step}")

## 📅 Part 9: Season Planning

### Long-term strategy

In [ ]:
# Season planner
planner = GameweekPlanner(current_gw=current_gw)

print("\n📅 SEASON PLANNING GUIDE")
print("=" * 80)

season_plan = planner.create_season_plan()
print(f"Current GW: {season_plan['current_gw']}")
print(f"Chips remaining: {', '.join(season_plan['chips_remaining'])}")
print("\nRecommended schedule:")
for gw, recommendation in season_plan['recommended_schedule'].items():
    print(f"  {gw}: {recommendation}")

print(f"\nTransfer strategy: {season_plan['transfer_strategy']}")
print(f"Captaincy: {season_plan['captaincy']}")

In [ ]:
# Weekly checklist
print("\n✅ WEEKLY PREPARATION CHECKLIST (GW" + str(current_gw) + ")")
print("=" * 80)

checklist = planner.weekly_checklist(current_gw)
for item in checklist:
    print(item)

## 🏆 Part 10: Mini-League Strategy

### Different approaches for leaders vs chasers

In [ ]:
# Mini-league strategy (customize with your rank)
your_rank = 5  # Your current rank in mini-league
total_players = 20  # Total players in mini-league

print("\n🏆 MINI-LEAGUE STRATEGY")
print("=" * 80)

ml_strategy = strategy.mini_league_strategy(your_rank, total_players)

print(f"Your rank: {ml_strategy['rank']}/{total_players} (Top {ml_strategy['rank_percentage']:.1f}%)")
print(f"Strategy type: {ml_strategy['strategy_type'].upper()}")
print(f"Approach: {ml_strategy['approach'].upper()}")
print(f"Risk level: {ml_strategy['risk_level']}")
print("\nRecommended tactics:")
for tactic in ml_strategy['tactics']:
    print(f"  ✓ {tactic}")

## 💰 Part 11: Bonus Point Prediction

### Players likely to get bonus points

In [ ]:
# Bonus point prediction
bonus_predictor = BonusPredictor(players_df)

print("\n💰 BONUS POINT PREDICTIONS")
print("=" * 80)

bonus_predictions = bonus_predictor.predict_bonus_potential()
top_bonus = bonus_predictions.head(20)

print("\nTop 20 Bonus Point Magnets:\n")
print(top_bonus.to_string(index=False))

print("\n💡 Target players with bonus_potential > 2.0 for consistent bonus points!")

## 📊 Part 12: Export All Metrics

### Save all calculations for further analysis

In [ ]:
# Export all metrics
print("\n💾 EXPORTING ALL METRICS")
print("=" * 80)

all_metrics = metrics.export_all_metrics(output_dir='./Data/')

print("\nAvailable metric DataFrames:")
for name, df in all_metrics.items():
    print(f"  {name}: {len(df)} rows")
    
print("\n✅ All metrics calculated and ready for analysis!")

## 🎓 Summary

### You now have access to:

1. ✅ **20+ Advanced Metrics** including xG, xA, xGI, BPS, PPM, ROI
2. ✅ **Position-Specific Analysis** for GKP, DEF, MID, FWD
3. ✅ **Captain Picks** based on multiple factors
4. ✅ **Differential Players** for gaining ranks
5. ✅ **Team Building Strategies** (Balanced, Template, Differential, Premium)
6. ✅ **Transfer Planning** with suggestions
7. ✅ **Chip Strategy** (Wildcard, BB, TC, FH timing)
8. ✅ **Season Planning** and weekly checklists
9. ✅ **Mini-League Strategy** (Leader/Chaser approaches)
10. ✅ **Bonus Point Predictions**

### Popular FPL YouTuber Techniques Implemented:

- 📺 **FPL Focal**: xGI analysis, underlying stats focus
- 📺 **Let's Talk FPL**: Captain picks, transfer planning
- 📺 **FPL Wire**: Differential strategy, template teams
- 📺 **FPL General**: Fixture analysis, form vs fixtures
- 📺 **FPL BlackBox**: Advanced metrics, value optimization
- 📺 **FPL Raptor**: Bonus point focus, BPS analysis

### Next Steps:

1. Customize the current_team list with your actual squad
2. Update current_gw with the current gameweek
3. Run weekly to get updated transfer suggestions
4. Use visualizations to identify value picks
5. Plan chips using the strategy guide

**Good luck with your FPL season! 🏆**